In [1]:
import pandas as pd
import numpy as np

All data are scrapped from BasketBall-Reference website

# **Data scrapping (from 1979-80 to 2020-21)**

In [2]:
#Seasons span
years = np.arange(1981,2022)

## **1. Classic stats per game**

In [3]:
'''Init'''

df_stats = pd.read_html('http://www.basketball-reference.com/leagues/NBA_1980_per_game.html'\
    , header = 0)[0]

#Traded players during the season
df_stats['No_trade'] = ~df_stats['Rk'].duplicated(keep=False)
df_stats = df_stats.drop_duplicates(['Rk'], keep='first')

#Adding some columns
df_stats = df_stats.assign(Season = "1979-1980", Decade = "1980", 
                Trade = (df_stats['No_trade']==False))
df_stats.Player = df_stats.Player.str.replace("*","")

#Dropping unusefull column
df_stats = df_stats.drop(columns=['No_trade'])

/var/folders/k3/3tz5rmjd3j3fsyyg9ynxk2680000gn/T/ipykernel_9656/2496363074.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_stats.Player = df_stats.Player.str.replace("*","")


In [4]:
'''Looping through seasons'''

#Account init
c=1

for year in years :
    url = f'http://www.basketball-reference.com/leagues/NBA_{year}_per_game.html'
    
    df2 = pd.read_html(url, header = 0)[0]
    
    #Traded players
    df2['No trade'] = ~df2['Rk'].duplicated(keep=False)
    df2 = df2.drop_duplicates(['Rk'], keep='first')

    #Adding same types of columns
    df2 = df2.assign(Season = f"{year-1}-{year}", Decade = f"{int(10*np.floor(year/10))}", 
                    Trade = (df2['No trade']==False))
    df2.Player = df2.Player.str.replace("*","")
    
    #Same dropping
    df2 = df2.drop(columns=['No trade'])
    
    #Concatenation
    df_stats = df_stats.append(df2, ignore_index = True)

    c+=1
    if c%10 == 0 :
        print(f"{c}/{years[-1]-years[0]+2}")
        print('-'*5)

#End
print(f"{c}/{years[-1]-years[0]+2}")
print('-'*5)
print('All data have been fetched.')

/var/folders/k3/3tz5rmjd3j3fsyyg9ynxk2680000gn/T/ipykernel_9656/166087837.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2.Player = df2.Player.str.replace("*","")


10/42
-----
20/42
-----
30/42
-----
40/42
-----
42/42
-----
All data have been fetched.


In [5]:
'''Columns types'''

df_stats = df_stats.drop(df_stats[df_stats.Rk == 'Rk'].index)

col_cat = ['Player', 'Pos', 'Tm','Season', 'Trade']

col_int =['Rk','Age','Decade','G','GS']

for column in df_stats.columns :
    if column not in col_cat and column not in col_int :
        if column not in col_int :
            df_stats[column] = df_stats[column].astype(float)
        else :
            df_stats[column] = df_stats[column].astype(int)

## **2. Advanced stats per game**

In [6]:
'''Init'''

df_adv = pd.read_html('https://www.basketball-reference.com/leagues/NBA_1980_advanced.html'\
    , header = 0)[0]

df_adv = df_adv.drop_duplicates(['Rk'], keep='first')

df_adv = df_adv.assign(Season = "1979-1980")
df_adv.Player = df_adv.Player.str.replace("*","")

/var/folders/k3/3tz5rmjd3j3fsyyg9ynxk2680000gn/T/ipykernel_9656/959464798.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_adv.Player = df_adv.Player.str.replace("*","")


In [7]:
'''Looping through seasons'''

c=1

for year in years :
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html'
    
    df2 = pd.read_html(url, header = 0)[0]
    
    df2 = df2.drop_duplicates(['Rk'], keep='first')

    df2 = df2.assign(Season = f"{year-1}-{year}")
    df2.Player = df2.Player.str.replace("*","")
    
    df_adv = df_adv.append(df2, ignore_index = True)

    c+=1
    if c%10 == 0 :
        print(f"{c}/{years[-1]-years[0]+2}")
        print('-'*5)

#End
print(f"{c}/{years[-1]-years[0]+2}")
print('-'*5)
print('All data have been fetched.')

/var/folders/k3/3tz5rmjd3j3fsyyg9ynxk2680000gn/T/ipykernel_9656/1528544668.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2.Player = df2.Player.str.replace("*","")


10/42
-----
20/42
-----
30/42
-----
40/42
-----
42/42
-----
All data have been fetched.


In [8]:
'''Columns types'''

df_adv = df_adv.drop(df_adv[df_adv.Rk == 'Rk'].index)

col_cat = ['Player', 'Pos', 'Tm','Season']

col_int =['Rk','Age','G']

for column in df_adv.columns :
    if column not in col_cat and column not in col_int :
        if column not in col_int :
            df_adv[column] = df_adv[column].astype(float)
        else :
            df_adv[column] = df_adv[column].astype(int)

empty_columns = [c for c in df_adv.columns if df_adv[c].isnull().all()]

df_adv.drop(empty_columns, axis='columns', inplace=True)

## **3. MVP trophy historic**

In [9]:
years_mvp = np.arange(1955,2022)

### a. Winners

In [10]:
df_mvp = pd.read_html('https://www.basketball-reference.com/awards/mvp.html', header = 1)[0]

In [11]:
seasons_mvp = []
for i in range(len(df_mvp)) :
    seasons_mvp.append(f'{years_mvp[i]}-{years_mvp[i+1]}')

In [12]:
df_mvp['Season'] = seasons_mvp[::-1]

df_mvp = df_mvp[['Season', 'Player']]

In [13]:
mvps = list(df_mvp.Player.values)
already_mvp = []
for i in range(len(mvps)) :
    already_mvp.append(mvps[i] in mvps[i+1:])

df_mvp['Already_mvp'] = already_mvp

### b. Vote results

In [14]:
df_vote_mvp = pd.read_html('https://www.basketball-reference.com/awards/awards_1980.html', header=1)[0]

In [15]:
usefull_columns = ['Player','Share']

df_vote_mvp = df_vote_mvp[usefull_columns]

df_vote_mvp = df_vote_mvp.assign(Season = '1979-1980')

In [16]:
'''Looping'''

c=1

for year in years :
    url = f'https://www.basketball-reference.com/awards/awards_{year}.html'
    
    df2 = pd.read_html(url, header = 1)[0]
    
    df2 = df2[usefull_columns]

    df2 = df2.assign(Season = f"{year-1}-{year}")

    df_vote_mvp = df_vote_mvp.append(df2, ignore_index = True)

    c+=1
    if c%10 == 0 :
        print(f"{c}/{years[-1]-years[0]+2}")
        print('-'*5)

#End
print(f"{c}/{years[-1]-years[0]+2}")
print('-'*5)
print('All data have been fetched.')

10/42
-----
20/42
-----
30/42
-----
40/42
-----
42/42
-----
All data have been fetched.


## **4. Collective results (standings)**

In [17]:
'''Init'''

L_df = pd.read_html('https://www.basketball-reference.com/leagues/NBA_1980_standings.html', header = 0)

In [18]:
def traitement(df) :
    if df.W.dtype == object :
        df.drop(df[df.W.str.isdigit() == False].index, inplace=True)
    df.rename({list(df.columns)[0]: 'Team'}, axis='columns', inplace=True)

    columns_to_convert = [c for c in df if c != 'Team']
    
    df.GB = df.GB.str.replace("—","0")

    for c in columns_to_convert :
        df[c] = pd.to_numeric(df[c], downcast='float')
      
    max_ws = max([int(w) for w in df.W])
    df['GBC'] = max_ws - df.W

    df['Playoffs'] = df.Team.str.endswith("*")
    df.Team = df.Team.str.replace("*","")
    df.sort_values(['W/L%','Playoffs'], ascending=[False, False], ignore_index=True, inplace=True)

    df['Ranking_Conf'] = df.index + 1

In [19]:
def traitement2(l_df, season) :
    df = l_df[0].append(l_df[1], ignore_index = True)
    df.sort_values(['W/L%','Playoffs'], ascending=[False, False], ignore_index=True, inplace=True)
    df['Ranking_League'] = df.index + 1

    max_ws = max([int(w) for w in df.W])
    df.GB = max_ws - df.W
    df["Season"] = season
    return df

In [20]:
for df in L_df :
    traitement(df)

/var/folders/k3/3tz5rmjd3j3fsyyg9ynxk2680000gn/T/ipykernel_9656/3127126252.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.Team = df.Team.str.replace("*","")


In [21]:
df_standings = traitement2(L_df, '1979-1980')

In [22]:
'''Looping'''
c=1

for year in years :
    url2 = f'https://www.basketball-reference.com/leagues/NBA_{year}_standings.html'
    
    L_df2 = pd.read_html(url2, header = 0)

    for df in L_df2 :
        traitement(df)
    
    df2 = traitement2(L_df2, f"{year-1}-{year}")
    
    df_standings = df_standings.append(df2, ignore_index = True)

    c+=1
    if c%10 == 0 :
        print(f"{c}/{years[-1]-years[0]+2}")
        print("-"*5)

#End
print(f"{c}/{years[-1]-years[0]+2}")
print("-"*5)
print('All data have been fetched.')
    

/var/folders/k3/3tz5rmjd3j3fsyyg9ynxk2680000gn/T/ipykernel_9656/3127126252.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.Team = df.Team.str.replace("*","")


10/42
-----
20/42
-----
30/42
-----
40/42
-----
42/42
-----
All data have been fetched.


## **5. Dataset construction**

In [23]:
df_stats.drop(list(df_stats.columns)[0], axis = 'columns', inplace = True)
df_adv.drop(list(df_adv.columns)[0], axis = 'columns', inplace = True)

### a. Add advanced stats to classic stats Dataframe

In [24]:
df_glob = df_stats.merge(df_adv, on = ['Player', 'Season', 'Age', 'Tm', 'G', 'Pos'])

In [25]:
df_glob = df_glob.rename(columns={'MP_x': 'MPG', 'MP_y': 'MPTot'})

### b. Adding MVPs

In [26]:
verif_glob = np.array([list(df_glob.Season.values),list(df_glob.Player.values)])
verif_glob = np.transpose(verif_glob)
verif_glob = [list(elt) for elt in verif_glob]

In [27]:
verif_mvp = np.array([list(df_mvp.Season.values),list(df_mvp.Player.values)])
verif_mvp = np.transpose(verif_mvp)
verif_mvp = [list(elt) for elt in verif_mvp]

verif_past_mvp = np.array([list(df_mvp[df_mvp.Already_mvp].Season.values),list(df_mvp[df_mvp.Already_mvp].Player.values)])
verif_past_mvp = np.transpose(verif_past_mvp)
verif_past_mvp = [list(elt) for elt in verif_past_mvp]

In [28]:
MVP, PastMVP = [],[]

for elt in verif_glob :
    MVP.append(elt in verif_mvp)
    PastMVP.append(elt in verif_past_mvp)

In [29]:
MVP, PastMVP = [],[]

for elt in verif_glob :
    MVP.append(elt in verif_mvp)
    if elt in verif_past_mvp :
        PastMVP.append(True)
    elif elt[1] in list(df_mvp.Player.values) :
        if elt[0] > min(list(df_mvp[df_mvp.Player == elt [1]].Season.values)) :
            PastMVP.append(True)
        else :
            PastMVP.append(False)
    else :
        PastMVP.append(False)

In [30]:
df_glob['MVP'] = MVP
df_glob['PastMVP'] = PastMVP

### c. Team's accronyms

In this part we are going to relate the team acronyms with their full name. Indeed, in the DataFrame regrouping the individual stats for players, the team is indicated with an acronym like 'GSW' for the 'Golden State Warriors' for example. However in the DataFrame gathering the team statistics (wins, PTS, REB, ...), the team is filled by its full name, hence the need to cross the two.

In the teams, only one has 2 accronyms : the 'Charlotte Hornets' : 'CHO' and 'CHH', that's why we decide to replace 'CHO' by 'CHH' in all the DataFrame of the individual statistics.

In [31]:
df_glob.Tm = df_glob.Tm.str.replace('CHO', 'CHH')

In [32]:
teams_acro = list(set(list(df_glob.Tm.values)))
teams_name = list(set(list(df_standings.Team.values)))
teams_acro.remove('TOT')
teams_acro.sort()
teams_name.sort()

tab_teams ={}
for i in range(len(teams_name)):
    tab_teams[teams_acro[i]]=teams_name[i]

tab_teams

{'ATL': 'Atlanta Hawks',
 'BOS': 'Boston Celtics',
 'BRK': 'Brooklyn Nets',
 'CHA': 'Charlotte Bobcats',
 'CHH': 'Charlotte Hornets',
 'CHI': 'Chicago Bulls',
 'CLE': 'Cleveland Cavaliers',
 'DAL': 'Dallas Mavericks',
 'DEN': 'Denver Nuggets',
 'DET': 'Detroit Pistons',
 'GSW': 'Golden State Warriors',
 'HOU': 'Houston Rockets',
 'IND': 'Indiana Pacers',
 'KCK': 'Kansas City Kings',
 'LAC': 'Los Angeles Clippers',
 'LAL': 'Los Angeles Lakers',
 'MEM': 'Memphis Grizzlies',
 'MIA': 'Miami Heat',
 'MIL': 'Milwaukee Bucks',
 'MIN': 'Minnesota Timberwolves',
 'NJN': 'New Jersey Nets',
 'NOH': 'New Orleans Hornets',
 'NOK': 'New Orleans Pelicans',
 'NOP': 'New Orleans/Oklahoma City Hornets',
 'NYK': 'New York Knicks',
 'OKC': 'Oklahoma City Thunder',
 'ORL': 'Orlando Magic',
 'PHI': 'Philadelphia 76ers',
 'PHO': 'Phoenix Suns',
 'POR': 'Portland Trail Blazers',
 'SAC': 'Sacramento Kings',
 'SAS': 'San Antonio Spurs',
 'SDC': 'San Diego Clippers',
 'SEA': 'Seattle SuperSonics',
 'TOR': 'Toron

It can be seen that the misplaced acronyms are :
- NOK and NOP to be reversed
- WAS and WSB to be reversed

In [33]:
tab_teams['NOK'] = 'New Orleans/Oklahoma City Hornets'
tab_teams['NOP'] = 'New Orleans Pelicans'
tab_teams['WAS'] = 'Washington Wizards'
tab_teams['WSB'] = 'Washington Bullets'

In [34]:
names = []
for acro in list(df_glob.Tm.values) :
    if acro != 'TOT' :
        names.append(tab_teams[acro])
    else :
        names.append('Trade')

df_glob.insert(4, 'Team', names)

### d. Adding collective results to global dataframe

In [35]:
df_glob = df_glob.merge(df_standings, on=['Team', 'Season'], how='outer')

### e. Adding MVP voting shares to global dataframe

In [36]:
df_glob = df_glob.merge(df_vote_mvp, on=['Player', 'Season'], how='outer')

In [37]:
df_glob.Share = df_glob.Share.fillna(0)

### f. Last details

We add the proportion of game played : $Game\ played\ prop = \frac {Game Played}{Team Wins + Team Loses}$

In [38]:
df_glob.G = df_glob.G.astype(int)

df_glob.W = df_glob.W.fillna(0)
df_glob.L = df_glob.L.fillna(0)
df_glob.W = df_glob.W.astype(int)
df_glob.L = df_glob.L.astype(int)

In [39]:
df_glob['Game_played_prop'] = df_glob[df_glob.Team != 'Trade'].G / (df_glob[df_glob.Team != 'Trade'].W + df_glob[df_glob.Team != 'Trade'].L)

In [40]:
df_glob.rename(columns = {'Share' : 'MVP_share'}, inplace = True)

We also add the boolean columns ```ShareYN``` which represent the fact that one player received votes for MVP or not.

In [41]:
df_glob = df_glob.assign(ShareYN = df_glob['MVP_share'] > 0)

## **6. Dataset download**

In [42]:
df_glob = df_glob[df_glob.Trade == False]

In [43]:
df_glob.to_csv('global.csv', index=False)

# **Data scrapping (2021-22 season, by the 2nb of February)**

## **1. Classic stats per game**

In [44]:
df_stats_22 = pd.read_html('http://www.basketball-reference.com/leagues/NBA_2022_per_game.html'\
    , header = 0)[0]

df_stats_22['No_trade'] = ~df_stats_22['Rk'].duplicated(keep=False)
df_stats_22 = df_stats_22.drop_duplicates(['Rk'], keep='first')


df_stats_22 = df_stats_22.assign(Season = "2021-2022", Decade = "2020", 
                            Trade = (df_stats_22['No_trade']==False))
df_stats_22.Player = df_stats_22.Player.str.replace("*","")

df_stats_22 = df_stats_22.drop(columns=['No_trade'])

/var/folders/k3/3tz5rmjd3j3fsyyg9ynxk2680000gn/T/ipykernel_9656/1534445803.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_stats_22.Player = df_stats_22.Player.str.replace("*","")


In [45]:
df_stats_22 = df_stats_22.drop(df_stats_22[df_stats_22.Rk == 'Rk'].index)

col_cat = ['Player', 'Pos', 'Tm','Season', 'Trade']

col_int =['Rk','Age','Decade','G','GS']


for column in df_stats_22.columns :
    if column not in col_cat and column not in col_int :
        if column not in col_int :
            df_stats_22[column] = df_stats_22[column].astype(float)
        else :
            df_stats_22[column] =df_stats_22[column].astype(int)

## **2. Advanced stats per game**

In [46]:
df_adv_22 = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2022_advanced.html'\
    , header = 0)[0]

df_adv_22 = df_adv_22.drop_duplicates(['Rk'], keep='first')

df_adv_22 = df_adv_22.assign(Season = "2021-2022")
df_adv_22.Player = df_adv_22.Player.str.replace("*","")

/var/folders/k3/3tz5rmjd3j3fsyyg9ynxk2680000gn/T/ipykernel_9656/3754823081.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_adv_22.Player = df_adv_22.Player.str.replace("*","")


In [47]:
df_adv_22 = df_adv_22.drop(df_adv_22[df_adv_22.Rk == 'Rk'].index)

col_cat = ['Player', 'Pos', 'Tm','Season']

col_int =['Rk','Age','G','GS']

for column in df_adv_22.columns :
    if column not in col_cat and column not in col_int :
        if column not in col_int :
            df_adv_22[column] = df_adv_22[column].astype(float)
        else :
            df_adv_22[column] = df_adv_22[column].astype(int)

empty_columns = [c for c in df_adv_22.columns if df_adv_22[c].isnull().all()]

df_adv_22.drop(empty_columns, axis='columns', inplace=True)

## **3. Collective results (standings)**

In [48]:
L_df_22 = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2022_standings.html', header = 0)[:2]

In [49]:
def traitement_22(df) :
    if df.W.dtype == object :
        df.drop(df[df.W.str.isdigit() == False].index, inplace=True)
    df.rename({list(df.columns)[0]: 'Team'}, axis='columns', inplace=True)

    columns_to_convert = [c for c in df if c != 'Team']
    
    df.GB = df.GB.str.replace("—","0")

    for c in columns_to_convert :
        df[c] = pd.to_numeric(df[c], downcast='float')
      
    max_ws = max([int(w) for w in df.W])
    df['GBC'] = max_ws - df.W
    
    df.Team[df.Team.str.endswith(' ')] = df[df.Team.str.endswith(' ')].Team.str[:-1]

    df.sort_values(['W/L%'], ascending=False, ignore_index=True, inplace=True)

    df['Ranking_Conf'] = df.index + 1

    df['Playoffs'] = df.Ranking_Conf <= 8

In [50]:
def traitement2_22(l_df, season) :
    df = l_df[0].append(l_df[1], ignore_index = True)
    df.sort_values(['W/L%'], ascending=False, ignore_index=True, inplace=True)
    df['Ranking_League'] = df.index + 1

    max_ws = max([int(w) for w in df.W])
    df.GB = max_ws - df.W
    df["Season"] = season
    return df

In [51]:
for df in L_df_22 :
    traitement_22(df)

/var/folders/k3/3tz5rmjd3j3fsyyg9ynxk2680000gn/T/ipykernel_9656/718517820.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Team[df.Team.str.endswith(' ')] = df[df.Team.str.endswith(' ')].Team.str[:-1]


In [52]:
df_standings_22 = traitement2_22(L_df_22, '2021-2022')

## **4. Dataset construction**

In [53]:
df_mvp_22 = df_mvp.drop(columns=['Already_mvp'])

already_mvp = set(df_mvp_22.Player.values)

In [54]:
df_stats_22.drop(list(df_stats_22.columns)[0], axis = 'columns', inplace = True)
df_adv_22.drop(list(df_adv_22.columns)[0], axis = 'columns', inplace = True)

In [55]:
PastMVP = []

for player in df_stats_22.Player.values :
    PastMVP.append(player in already_mvp)

df_stats_22['PastMVP'] = PastMVP

In [56]:
df_22 = df_stats_22.merge(df_adv_22, on = ['Player', 'Season', 'Age', 'Tm', 'G', 'Pos'])

df_22 = df_22.rename(columns={'MP_x': 'MPG', 'MP_y': 'MPTot'})

df_22 = df_22[df_22.Trade == False]

In [57]:
df_22.Tm = df_22.Tm.str.replace('CHO', 'CHH')

In [58]:
teams_acro_22 = list(set(list(df_22.Tm.values)))
teams_name_22 = list(set(list(df_standings_22.Team.values)))
teams_acro_22.sort()
teams_name_22.sort()

tab_teams_22 ={}
for i in range(len(teams_name_22)):
    tab_teams_22[teams_acro_22[i]]=teams_name_22[i]

In [59]:
names_22 = []
for acro in list(df_22.Tm.values) :
    if acro != 'TOT' :
        names_22.append(tab_teams_22[acro])
    else :
        names_22.append('Trade')

df_22.insert(4, 'Team', names_22)

In [60]:
df_22 = df_22.merge(df_standings_22, on=['Team', 'Season'], how='outer')

In [61]:
df_22['G'] = df_22['G'].astype(int)

In [62]:
df_22['Game_played_prop'] = df_22.G/(df_22.W + df_22.L)

In [63]:
df_22.to_csv('2022.csv', index = False)